# Setup

In [1]:
import os
os.environ['TOKEN'] = '#BEARER TOKEN HERE'
env_label = '#ENVIROMENT LABEL HERE'

In [3]:
import requests 
import pandas as pd 
import time

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
headers = create_headers(os.environ['TOKEN'])

# Download data


In [65]:
def create_url(keyword, start_date, end_date, env_label, endpoint="30day"):
    # endpoint can be "30day" or "fullarchive"
    
    search_url = "https://api.twitter.com/1.1/tweets/search/{}.json".format(endpoint+"/"+env_label) 
    # change params based on the endpoint you are using
    query_params = {'query': keyword, 'fromDate': start_date, 'toDate': end_date}
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token is not None and next_token != '':
      params['next'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def get_data(keyword, start_time, end_time, next_token, env_label, endpoint):
  results = []
  
  while next_token is not None:
    ##this part here for one request
    url = create_url(keyword, start_time,end_time, env_label, endpoint)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    
    if "results" in json_response:
      results.extend(json_response["results"])
    ### up until this point
    if "next" in json_response:
        next_token = json_response["next"]
    else:
      next_token = None
    time.sleep(2)
  
  return results


def get_single_response(keyword, start_time, end_time, env_label, endpoint):
  # endpoint can be fullarchive or 30day
  results = []
  url = create_url(keyword, start_time,end_time, env_label, endpoint)
  json_response = connect_to_endpoint(url[0], headers, url[1])
  
  if "results" in json_response:
    results.extend(json_response["results"])
  
  return results

In [91]:
# Request parameters

# if you are using the 30day endpoint, make sure you specify dates that are within 30day range!
start_date = None # "202110270000"
end_date = None # "202111250000"

endpoint = "30day"

# keyword = "from:BBCWorld covid lang:en"
# keyword = "from:CNN covid lang:en"
# keyword = "from:AP covid lang:en"
# keyword = "from:Reuters covid lang:en"
# keyword = "from:XHNews covid lang:en"
keyword = "from:CGTNOfficial covid lang:en"

# result_type=popular
# is:verified

In [85]:
# IMPORTANT: run this cell only if you really need it
tweets = get_single_response(keyword, start_date, end_date, env_label, endpoint)

In [92]:
all_tweets = get_data(keyword, start_date, end_date, '', env_label, endpoint)

In [93]:
all_tweets_df = pd.DataFrame(all_tweets)

In [94]:
all_tweets_df.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,extended_tweet,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,retweeted_status
0,Mon Nov 29 07:40:00 +0000 2021,1465224005963722753,1465224005963722753,The newly discovered COVID-19 variant #Omicron...,"[0, 140]","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,None,NaN,None,None,"{'id': 1115874631, 'id_str': '1115874631', 'na...",None,None,None,None,False,{'full_text': 'The newly discovered COVID-19 v...,0,1,3,16,"{'hashtags': [{'text': 'Omicron', 'indices': [...",False,False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN
1,Mon Nov 29 02:40:00 +0000 2021,1465148508550287363,1465148508550287363,The Chinese mainland recorded 41 confirmed COV...,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,None,NaN,None,None,"{'id': 1115874631, 'id_str': '1115874631', 'na...",None,None,None,None,False,{'full_text': 'The Chinese mainland recorded 4...,0,0,11,17,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN
2,Mon Nov 29 02:00:01 +0000 2021,1465138447321124869,1465138447321124869,It is not yet clear if the #Omicron variant is...,"[0, 140]","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,None,NaN,None,None,"{'id': 1115874631, 'id_str': '1115874631', 'na...",None,None,None,None,False,{'full_text': 'It is not yet clear if the #Omi...,0,0,5,13,"{'hashtags': [{'text': 'Omicron', 'indices': [...",False,False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN
3,Mon Nov 29 00:03:31 +0000 2021,1465109127277924356,1465109127277924356,COVID-19: S. African president warns of fourth...,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,"{'id': 1115874631, 'id_str': '1115874631', 'na...",None,None,None,None,False,NaN,2,1,7,25,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,False,low,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN
4,Sun Nov 28 21:31:09 +0000 2021,1465070786331488260,1465070786331488260,The COVID-19 #Omicron variant is sparking glob...,"[0, 140]","<a href=""https://about.twitter.com/products/tw...",True,NaN,None,NaN,None,None,"{'id': 1115874631, 'id_str': '1115874631', 'na...",None,None,None,None,True,{'full_text': 'The COVID-19 #Omicron variant i...,0,2,6,3,"{'hashtags': [{'text': 'Omicron', 'indices': [...",False,False,False,low,en,[{'tag': None}],1.465070e+18,1465070494319788042,{'created_at': 'Sun Nov 28 21:30:00 +0000 2021...,"{'url': 'https://t.co/s9p1WnWVMQ', 'expanded':...",NaN,NaN


In [95]:
all_tweets_df.to_csv("CGTNOfficial_all_tweets_df.csv", index=False)

## Get Tweets from a specific user

API Reference: https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

BBC Twitter accounts: `BBCWorld`, `BBC`, `BBCPolitics`, `BBCBreaking`, `BBCNews` (UK), `BBCNewsAsia`, `BBCNewsNorthAmerica`


In [33]:
endpoint = "30day"

In [8]:
def create_user_url(screen_name, env_label, endpoint="30day"):
  user_url = "https://api.twitter.com/1.1/statuses/user_timeline.json" 
  # change params based on the endpoint you are using
  query_params = {'screen_name': screen_name}
  return (user_url, query_params)

def get_single_user_response(screen_name, env_label, endpoint):
  # endpoint can be fullarchive or 30day
  results = []
  url = create_user_url(screen_name, env_label, endpoint)
  json_response = connect_to_endpoint(url[0], headers, url[1])
  
  if "results" in json_response:
    results.extend(json_response["results"])
  
  return results


In [ ]:
# Request parameters
screen_name = "BBC"

In [62]:
# IMPORTANT: run this cell only if you really need it
BBC_tweets = get_single_user_response(screen_name, env_label, endpoint)

In [63]:
len(BBC_tweets)

0